In [ ]:
import pandas as pd
import time
from web3 import Web3

# 1. Load CSV data
df = pd.read_csv("Logistics-Data.csv")
print("📦 First few rows of the CSV file:")
print(df.head())

# 2. Connect to Ganache
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")
    exit(1)

# 3. Load the smart contract
contract_address = web3.to_checksum_address("0x1f97616ae12389cd3d6b59274f1b8b9d6cba3f2a")

abi = [
	{
		"inputs": [{"internalType": "address","name": "device","type": "address"}],
		"name": "authorizeDevice",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [{"internalType": "address","name": "device","type": "address"}],
		"name": "revokeDevice",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{"internalType": "string","name": "deviceID","type": "string"},
			{"internalType": "string","name": "dataType","type": "string"},
			{"internalType": "string","name": "value","type": "string"}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [{"internalType": "address","name": "","type": "address"}],
		"name": "authorizedDevices",
		"outputs": [{"internalType": "bool","name": "","type": "bool"}],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{"internalType": "string","name": "deviceID","type": "string"},
			{"internalType": "uint256","name": "index","type": "uint256"}
		],
		"name": "getDataByIndex",
		"outputs": [
			{"internalType": "string","name": "","type": "string"},
			{"internalType": "string","name": "","type": "string"},
			{"internalType": "uint256","name": "","type": "uint256"}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [{"internalType": "string","name": "deviceID","type": "string"}],
		"name": "getDataCount",
		"outputs": [{"internalType": "uint256","name": "","type": "uint256"}],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [{"internalType": "address","name": "","type": "address"}],
		"stateMutability": "view",
		"type": "function"
	}
]

# Load contract
contract = web3.eth.contract(address=contract_address, abi=abi)
web3.eth.default_account = web3.eth.accounts[0]

print(f"✅ Connected to Smart Contract at {contract_address}")
print(f"🔑 Using account: {web3.eth.default_account}")

# 4. Authorize sender account
device_to_authorize = web3.eth.default_account
try:
    auth_txn = contract.functions.authorizeDevice(device_to_authorize).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    web3.eth.wait_for_transaction_receipt(auth_txn)
    print(f"✅ Device {device_to_authorize} authorized successfully.")
except Exception as e:
    print(f"⚠️ Authorization may already exist. Proceeding...")

# 5. Send IoT data
def send_iot_data(device_id, data_type, data_value):
    txn = contract.functions.storeData(device_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"🛰️ Stored [{device_id}] - {data_type}: {data_value} | Txn Hash: {receipt.transactionHash.hex()}")

print("\n🔄 Sending IoT records to the blockchain...")
for _, row in df.iterrows():
    device_id = str(row["package_id"])
    send_iot_data(device_id, "temperature", str(row["temperature"]))
    send_iot_data(device_id, "humidity", str(row["humidity"]))
    send_iot_data(device_id, "shock", str(row["shock"]))
    time.sleep(1) 

# 6. Verify data
first_device_id = str(df.iloc[0]["package_id"])
record_count = contract.functions.getDataCount(first_device_id).call()
print(f"\n📊 Total records for device {first_device_id}: {record_count}")

if record_count > 0:
    data_type, value, timestamp = contract.functions.getDataByIndex(first_device_id, 0).call()
    print("\n📥 First record retrieved from blockchain:")
    print(f"Device ID: {first_device_id}")
    print(f"Data Type: {data_type}")
    print(f"Data Value: {value}")
    print(f"Timestamp: {timestamp}")
else:
    print("⚠️ No data found for the device.")


📦 First few rows of the CSV file:
                    timestamp package_id     origin destination  \
0  2025-04-29 12:08:30.336512   PKG85046  São Paulo      Moscow   
1  2025-04-29 13:17:30.336512   PKG59811      Tokyo       Seoul   
2  2025-04-29 14:36:30.336512   PKG80313  Cape Town       Seoul   
3  2025-04-29 15:23:30.336512   PKG90642      Tokyo   São Paulo   
4  2025-04-29 16:06:30.336512   PKG86797     Moscow      Berlin   

            location closest_city      status  temperature  humidity  shock  
0   33.3012, 13.5857        Cairo  In Transit         26.0      51.6   0.20  
1  36.7251, 136.1661        Tokyo  In Transit         11.7      84.9   0.20  
2   24.1418, 107.766      Bangkok  In Transit         34.1      83.4   0.04  
3    3.6758, 36.9013        Cairo  In Transit         21.8      49.2   0.48  
4   52.3556, 18.2952       Berlin  In Transit         13.9      81.2   0.37  
✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0x1F97616Ae12389Cd3D6b592